## PyTorch MNIST CNN Example on Jetson

In [1]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import numpy as np
#import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
#import matplotlib.pyplot as plt

/home/jwolf/torch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
image_path = './'

transform = transforms.Compose([transforms.ToTensor()])

mnist_dataset = torchvision.datasets.MNIST(
    root=image_path, train=True,
    transform=transform, download=True
)

100.0%


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100.0%


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100.0%


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100.0%

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [4]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
mnist_valid_dataset = Subset(mnist_dataset,
                             torch.arange(10000))

mnist_train_dataset = Subset(mnist_dataset,
                             torch.arange(
                                 10000,
                                 len(mnist_dataset)
                             ))

mnist_test_dataset = torchvision.datasets.MNIST(
    root=image_path, train=False,
    transform=transform, download=False
)

In [6]:
batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset,
                      batch_size,
                      shuffle=True)
valid_dl = DataLoader(mnist_valid_dataset,
                      batch_size,
                      shuffle=False)

In [7]:
model = nn.Sequential()
model.add_module(
    'conv1',
    nn.Conv2d(
        in_channels=1, out_channels=32,
        kernel_size=5, padding=2
    )
)
# Layer 2 reLU
model.add_module('relu1', nn.ReLU())

# Layer 3 Pooling
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))

# Layer 4 CNN
model.add_module(
    'conv2',
    nn.Conv2d(
        in_channels=32, out_channels=64,
        kernel_size=5, padding=2
    )
)

# Layer 5 reLU
model.add_module('relu2', nn.ReLU())

# Layer 6 Pooling
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))

# Layer 7 Flatten
model.add_module('flatten', nn.Flatten())
# x = torch.ones((4, 1, 28, 28))
# print(model(x).shape)

# Layer 8 Fully Connected
model.add_module('fc1', nn.Linear(3136, 1024))

# Layer 9 reLU
model.add_module('relu3', nn.ReLU())

# Layer 10 Dropout
model.add_module('dropout', nn.Dropout(p=0.5))

# Layer 11 Fully Connected
model.add_module('fc2', nn.Linear(1024, 10))

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model = model.to(device)
loss_fn = loss_fn.to(device)

Using cuda device


In [9]:
def train(model, num_epochs, train_dl, valid_dl):
    loss_hist_train = [0] * num_epochs
    accuracy_hist_train = [0] * num_epochs
    loss_hist_valid = [0] * num_epochs
    accuracy_hist_valid = [0] * num_epochs

    for epoch in range(num_epochs):
        model.train()
        for x_batch, y_batch in train_dl:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_hist_train[epoch] += loss.item() * y_batch.size(0)
            is_correct = (
                    torch.argmax(pred, dim=1) == y_batch
            ).float()
            accuracy_hist_train[epoch] += is_correct.sum()
        loss_hist_train[epoch] /= len(train_dl.dataset)
        accuracy_hist_train[epoch] /= len(train_dl.dataset)

        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)
                loss_hist_valid[epoch] += loss.item() * y_batch.size(0)
                is_correct = (
                        torch.argmax(pred, dim=1) == y_batch
                ).float()
                accuracy_hist_valid[epoch] += is_correct.sum()
        loss_hist_valid[epoch] /= len(valid_dl.dataset)
        accuracy_hist_valid[epoch] /= len(valid_dl.dataset)

        print(f'Epoch {epoch + 1} accuracy: '
              f'{accuracy_hist_train[epoch]:.4f} val_accuracy: '
              f'{accuracy_hist_valid[epoch]:.4f}')
    return loss_hist_train, loss_hist_valid, \
        accuracy_hist_train, 

In [10]:
torch.manual_seed(1)
num_epochs = 10
hist = train(model, num_epochs, train_dl, valid_dl)

Epoch 1 accuracy: 0.9504 val_accuracy: 0.9810
Epoch 2 accuracy: 0.9837 val_accuracy: 0.9865
Epoch 3 accuracy: 0.9890 val_accuracy: 0.9828
Epoch 4 accuracy: 0.9916 val_accuracy: 0.9884
Epoch 5 accuracy: 0.9933 val_accuracy: 0.9885
Epoch 6 accuracy: 0.9949 val_accuracy: 0.9899
Epoch 7 accuracy: 0.9953 val_accuracy: 0.9896
Epoch 8 accuracy: 0.9956 val_accuracy: 0.9881
Epoch 9 accuracy: 0.9962 val_accuracy: 0.9895
Epoch 10 accuracy: 0.9969 val_accuracy: 0.9900


In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### Conv2D Shape:
<code>torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)<code>
<br>
<code>nn.Conv2d(in_channels, out_channels, kernel_size, padding)<code>    
- N: batch size
- C: number of channels
- H: height of input planes in pixels
- W: width in pixels
<br>
- Input: (N, Cin, Hin, Win) or (Cin, Hin, Win)
- Output: (n, Cout, Hout, Wout) or (Cout, Hout, Wout)
<br>
- Hout = ((Hin + 2 * padding - dilation x (kernel_size -1) -1) / stride) + 1
- Wout = ((Win + 2 * padding - dilation x (kernel_size -1) -1) / stride) + 1

In [3]:
((28 + 2 * 2 - 1 * (5-1)-1)/2)+1

14.5